## Informações Gerais
| Informações | Detalhes |
| ----------- | -------- |
| Nome Tabela | gold.venda_total_estabelecimentos |
| Origem | silver.pedidos |

## Historico de atualização
| Data | Desenvolvido por | Motivo |
| ----------- | -------- | -------|
| 06/10/2024 | Matheus | Criação do Notebook |

In [0]:
from pyspark.sql.functions import current_date, current_timestamp, expr

In [0]:
database = "gold"
tabela = "venda_total_produto"

In [0]:
#Funções
# Função que aplica os comentarios na tabela
def adicionaComentariosTabela(a, b, c, d):
    spark.sql(f"COMMENT ON TABLE {a}.{b} IS '{c}'")
    for key, value in d.items():
        sqlaux = f"ALTER TABLE {a}.{b} CHANGE COLUMN {key} COMMENT '{value}'"
        spark.sql(sqlaux)

In [0]:
df_pedidos = spark.sql(
    f"""
    SELECT
        produto AS nome_produto,
        CAST(SUM(quantidade) AS INT) AS quantidade_total,
        CAST(SUM(preco) AS DECIMAL(20,2)) AS valor_total
    FROM
        silver.pedidos
    GROUP BY ALL
    """
)

In [0]:
df_pedidos = df_pedidos.withColumn("data_carga", current_date())
df_pedidos = df_pedidos.withColumn("data_hora_carga", expr("current_timestamp() - INTERVAL 3 HOURS"))

In [0]:
df_pedidos.display()

nome_produto,quantidade_total,valor_total,data_carga,data_hora_carga
Suco Natural,12931,444.03,2024-10-06,2024-10-06T19:35:22.516Z
Vinho ABC,14191,614.80,2024-10-06,2024-10-06T19:35:22.516Z
Refrigerante 123,11741,459.77,2024-10-06,2024-10-06T19:35:22.516Z
Cerveja XYZ,15036,606.78,2024-10-06,2024-10-06T19:35:22.516Z
Água Mineral,12874,533.71,2024-10-06,2024-10-06T19:35:22.516Z


In [0]:
comentario_tabela = 'Entidade os produtos mais vendidos por estabelecimento'

lista_comentario_colunas = {
    'nome_produto': 'Nome do produto.',
    'quantidade_total': 'Quantidade total vendida.',
    'valor_total': 'Valor total vendido.',
    'data_carga': 'Data que o registro foi carregado.',
    'data_hora_carga': 'Data e hora que o registro foi carregado.'
}


In [0]:
df_pedidos.write \
    .format('delta') \
    .mode('overwrite') \
    .clusterBy("nome_produto") \
    .option('overwriteSchema', 'true') \
    .saveAsTable(f'{database}.{tabela}')
adicionaComentariosTabela(database, tabela, comentario_tabela, lista_comentario_colunas)
print("Comentarios adicionados nas tabelas")
print("Dados gravados com sucesso!")

Comentarios adicionados nas tabelas
Dados gravados com sucesso!


In [0]:
spark.sql(f"OPTIMIZE {database}.{tabela}")
print("Processo de otmização finalizado!")

Processo de otmização finalizado!


In [0]:
%sql
select * 
from 
gold.venda_total_produto

nome_produto,quantidade_total,valor_total,data_carga,data_hora_carga
Suco Natural,12931,444.03,2024-10-06,2024-10-06T19:36:19.239Z
Vinho ABC,14191,614.80,2024-10-06,2024-10-06T19:36:19.239Z
Refrigerante 123,11741,459.77,2024-10-06,2024-10-06T19:36:19.239Z
Cerveja XYZ,15036,606.78,2024-10-06,2024-10-06T19:36:19.239Z
Água Mineral,12874,533.71,2024-10-06,2024-10-06T19:36:19.239Z


Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.